In [ ]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
from tvDatafeed import TvDatafeed, Interval
import multiprocessing
import time
import glob

In [108]:
def get_nasq_syms():
  with open("watchlist_NASDAQ.txt", "r") as f:
    lines = f.readlines()
    f.close()
  line = lines[0].split(",")
  symbols = []
  for l in line:
    x = l.split(":")
    symbols.append(x[1])
  return symbols
def download_data_p(tv,sym,exc,tf):
    data =tv.get_hist(sym,exchange=exc, interval=tf, n_bars=50000)
    if data.empty:
      pass
    else:
         try:
           if np.array(data).shape[0] > 15:
             data.to_csv(f"/content/data/{sym}.csv")
         except:
            pass
def download_data(symbols):
  work_with_dir()
  tv = TvDatafeed()
  for sym in symbols:
    print(f'{symbols.index(sym)+1}/{len(symbols)} ',end=" ")
    p = multiprocessing.Process(target=download_data_p, name="dd", args=(tv,sym,"KUCOIN",Interval.in_1_hour))
    p.start()
    time.sleep(2)
    p.terminate()
    p.join()
def get_tbinances():
  with open("bitr.txt","r") as f:
     lines = f.readlines()
  nl = []
  for line in lines:
    nl.append(line.strip().split(":")[1])
  return nl
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["buy","sell"], axis=1)
  y = data[["buy","sell"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        unattached_dfs.append(process(pd.read_csv(address)))
     nud = []
     for i in unattached_dfs:
       if i.size != 0:
         nud.append(i)
     data = np.concatenate(nud)
     unattached_dfs = []
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")   

In [ ]:
symbols = get_tbinances()
print(f"Symbols: {len(symbols)}")
download_data(symbols,)

Symbols: 1143
Data Folder Removed
1/1143  2/1143  3/1143  4/1143  5/1143  6/1143  7/1143  8/1143  9/1143  10/1143  11/1143  12/1143  13/1143  14/1143  15/1143  16/1143  17/1143  18/1143  19/1143  20/1143  21/1143  22/1143  23/1143  24/1143  25/1143  26/1143  27/1143  28/1143  29/1143  30/1143  31/1143  32/1143  33/1143  34/1143  35/1143  36/1143  37/1143  38/1143  39/1143  40/1143  41/1143  42/1143  43/1143  44/1143  45/1143  46/1143  47/1143  48/1143  49/1143  50/1143  51/1143  52/1143  53/1143  54/1143  55/1143  56/1143  57/1143  58/1143  59/1143  60/1143  61/1143  62/1143  63/1143  64/1143  65/1143  66/1143  67/1143  68/1143  69/1143  70/1143  71/1143  72/1143  73/1143  74/1143  75/1143  76/1143  77/1143  78/1143  79/1143  80/1143  81/1143  82/1143  83/1143  84/1143  85/1143  86/1143  87/1143  88/1143  89/1143  90/1143  91/1143  92/1143  93/1143  94/1143  95/1143  96/1143  97/1143  98/1143  99/1143  100/1143  101/1143  102/1143  103/1143  104/1143  105/1143  106/1143  107/1143  108/

In [100]:
clmns = list(range(1, 61))
clmns.append("suggestion")
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-10,10))
    row = scaler.fit_transform(row)
    return row
def make_y(data,i):
  sugg = "sell"
  if data[i][3] > data[i-1][3]:
      sugg = "buy"
  return sugg
def process(data):
    data = data.dropna()
    row = []
    data.drop("symbol",axis=1,inplace=True)
    data.drop("datetime",axis=1,inplace=True)
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(31, data.shape[0]-1):
        grow = []
        ggggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][0] - data[i-(1+x)][0]] )
            ggggrow.append([data[i-x][3] - data[i-(1+x)][3]] )
        arr = np.array(grow).flatten()
        arr4 = np.array(ggggrow).flatten()
        sugg = make_y(data,i)
        arr = np.concatenate((arr, arr4), axis=0).reshape(-1,1)
        arr = scaler(arr.reshape(-1, 1))
        arr = np.append(arr, sugg)
        row.append(arr)
    grow = []
    ggrow = []
    gggrow = []
    arr = []
    return np.array(row)

In [ ]:
folder_name = extract_data(200)
to_par(folder_name,100)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

In [ ]:
model = Sequential()

model.add(Dense(512, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

opt = tf.keras.optimizers.Adamax()

model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(xTrain,yTrain,epochs=25,batch_size=128,validation_data=(xTest,yTest))
acr = str(round(model.evaluate(xTest,yTest)[1],4)).replace("0.","")
model.save(f"TMSG127_{acr}.h5")

In [59]:
def process_for_prediction(data,index):
        i = index
        if "symbol" in data.columns:
              data.drop("symbol",axis=1,inplace=True) 
        if "datetime" in data.columns:
              data.drop("datetime",axis=1,inplace=True)
        if "Adj Close" in data.columns:
              data.drop("Adj Close",axis=1,inplace=True)

        data = np.array(data)
        grow = []
        ggggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][0] - data[i-(1+x)][0]] )
            ggggrow.append([data[i-x][3] - data[i-(1+x)][3]] )
        arr = np.array(grow).flatten()
        arr4 = np.array(ggggrow).flatten()
        sugg = make_y(data,i)
        arr = np.concatenate((arr, arr4), axis=0).reshape(-1,1)
        arr = scaler(arr.reshape(-1, 1))
        return arr
def make_prediction_for_yf(symbol,period,timeframe,index):
    raw_data = process_for_prediction(yf.download(symbol,period=period,interval=timeframe),index)
    return f"YF  : {model.predict(np.array(raw_data).reshape(1,-1))}"
def make_prediction_for_tv(symbol,exchange,timeframe,tindex):
   tv = TvDatafeed()
   raw_data = process_for_prediction(tv.get_hist(symbol=symbol,exchange=exchange,interval=timeframe,n_bars=35),tindex)
   return  f"TVB : {model.predict(np.array(raw_data).reshape(1,-1))}"
def make_prediction(ysymbol,period,timeframe,tsymbol,texchange,ttimeframe,tindex,index):
  print(make_prediction_for_yf(ysymbol,period,timeframe,index))
  print(make_prediction_for_tv(tsymbol,texchange,ttimeframe,tindex))

In [60]:
make_prediction("eth-usd","max","1mo","ethusd","bitstamp",Interval.in_monthly,-1,-2)

[*********************100%***********************]  1 of 1 completed


YF  : [[0.78266525 0.21543488]]
TVB : [[0.7848438  0.21402735]]
